# Retraining after first attempt to prove correct
**Important note:**
In the process of initiating the retraining we found a bug in the environment:
Essentially, the area which now turned out to be buggy has not been trained on originally as it never occurred in the training samples due to a buggy bounds check (mixup between obstacle size `c` and wind speed `w` in `is_in_bounds`

In [1]:
import gym
import time

In [2]:
import pickle
import numpy as np
import polytope as pc

In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import torch
from torch import nn

In [4]:
import zeppelin_gym.env3

In [5]:
torch.manual_seed(42)

In [6]:
env = gym.make('zeppelin-v3')

/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Environment '<class 'zeppelin_gym.env3.ZeppelinEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [7]:
env.unwrapped.FUEL_RESTRAINT = False
env.unwrapped.OBSTACLE_REWARD = -1000.
env.unwrapped.NO_FUEL_REWARD = 0.
# done reward = (FUEL_RESTRAINT) ? r+fuel*r : 2*r
env.unwrapped.DONE_REWARD = 0.5
env.unwrapped.TIME_STEP_REWARD = 0.
env.unwrapped.INCLUDE_UNWINNABLE = False

In [8]:
env.seed(42)

[42]

In [14]:
retrain_polytopes = None
with open("zeppelin-small2-polytopes.pickle","rb") as f:
    retrain_polytopes = pickle.load(f)

In [25]:
poly_region = pc.Region(retrain_polytopes)
poly_region.contains(np.array([[72.],[131.],[80.],[30.]]),abs_tol=20)

array([False])

In [10]:
# Total volume of polytopes?
# Share of instances usually (i.e. without "focus polytopes")

# Volume of state space:
N = 100000
n=4
l_b = np.array([env.unwrapped.MIN_X,env.unwrapped.MIN_Y,env.unwrapped.MIN_C,env.unwrapped.MAX_VELOCITY-env.unwrapped.MAX_TURBULENCE+0.1])
u_b = np.array([env.unwrapped.MAX_X,env.unwrapped.MAX_Y,env.unwrapped.MAX_C,env.unwrapped.MAX_WIND_SPEED])
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if not env.unwrapped.is_crash(x) and not env.unwrapped.reached_goal(x) and env.unwrapped.is_in_bounds(x):
        s+=1
print("s: ",s)
total_vol = np.prod(u_b - l_b) * (s / N)
print("total volume: ", total_vol)

poly_region = pc.Region(retrain_polytopes)
l_b, u_b = poly_region.bounding_box
l_b = l_b.flatten()
u_b = u_b.flatten()
print(l_b,",",u_b)
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if x in poly_region:
        if not env.unwrapped.is_crash(x) and not env.unwrapped.reached_goal(x) and env.unwrapped.is_in_bounds(x):
                s+=1
poly_vol = np.prod(u_b - l_b) * (s / N)
print("poly volume: ", poly_vol)

# We only have an upper bound for the share, since polytopes may be partially outside the state space of interest
poly_share = poly_vol/total_vol
print("max share: ", poly_share)

s:  4512
total volume:  50332262.4
[-268.98618  -97.5       10.         5.     ] , [400. 400.  80.  30.]
poly volume:  16197547.64683712
max share:  0.32181242953301303


In [31]:
eval_episode_length=30000
training_episode_length=100000

In [32]:
env.seed(42)
torch.manual_seed(42)

In [33]:
model = PPO.load("zeppelin-avoidance-windsystem-small2-1400000")
model.set_env(env)

env.init_polytopes(1.0,[])
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward:0.97 +/- 5.78


In [34]:
env.init_polytopes(0.0,retrain_polytopes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:0.03 +/- 31.11


In [35]:
results_overall = {}
results_polys = {}

In [36]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("zeppelin-avoidance-windsystem-small2-1400000")
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    env.unwrapped.INCLUDE_UNWINNABLE = False
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-100000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.21     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 365      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.98        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 314         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.027606804 |
|    clip_fraction        | 0.241       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.1        |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 286        |
|    iterations           | 11         |
|    time_elapsed         | 78         |
|    total_timesteps      | 22528      |
| train/                  |            |
|    approx_kl            | 0.02297163 |
|    clip_fraction        | 0.242      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0456     |
|    explained_variance   | 0.00217    |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0218     |
|    n_updates            | 6960       |
|    policy_gradient_loss | 0.00489    |
|    std                  | 0.238      |
|    value_loss           | 488        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.27        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 281         |
|    iterations           | 20          |
|    time_elapsed         | 145         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.020620484 |
|    clip_fraction        | 0.242       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0967      |
|    explained_variance   | 0.00664     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.027       |
|    n_updates            | 7050        |
|    policy_gradient_loss | 0.00687     |
|    std                  | 0.231       |
|    value_loss           | 1.4e+03     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.08        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 270         |
|    iterations           | 29          |
|    time_elapsed         | 219         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.018757604 |
|    clip_fraction        | 0.236       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0906      |
|    explained_variance   | 0.00462     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0197     |
|    n_updates            | 7140        |
|    policy_gradient_loss | 0.0052      |
|    std                  | 0.233       |
|    value_loss           | 972         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.17       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 269        |
|    iterations           | 38         |
|    time_elapsed         | 288        |
|    total_timesteps      | 77824      |
| train/                  |            |
|    approx_kl            | 0.01393988 |
|    clip_fraction        | 0.195      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0843     |
|    explained_variance   | 0.00512    |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0107     |
|    n_updates            | 7230       |
|    policy_gradient_loss | 0.00334    |
|    std                  | 0.231      |
|    value_loss           | 918        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.47        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 430         |
|    iterations           | 6           |
|    time_elapsed         | 28          |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.036565084 |
|    clip_fraction        | 0.233       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.144       |
|    explained_variance   | 0.000125    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0404      |
|    n_updates            | 6910        |
|    policy_gradient_loss | 0.0213      |
|    std                  | 0.225       |
|    value_loss           | 489         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.17        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 377         |
|    iterations           | 15          |
|    time_elapsed         | 81          |
|    total_timesteps      | 30720       |
| train/                  |             |
|    approx_kl            | 0.022794968 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.129       |
|    explained_variance   | -8.95       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0332      |
|    n_updates            | 7000        |
|    policy_gradient_loss | 0.00471     |
|    std                  | 0.227       |
|    value_loss           | 0.00479     |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.24       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 370        |
|    iterations           | 24         |
|    time_elapsed         | 132        |
|    total_timesteps      | 49152      |
| train/                  |            |
|    approx_kl            | 0.02523032 |
|    clip_fraction        | 0.217      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.106      |
|    explained_variance   | -130       |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0145    |
|    n_updates            | 7090       |
|    policy_gradient_loss | 0.00769    |
|    std                  | 0.23       |
|    value_loss           | 0.147      |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.34      |
|    ep_rew_mean          | 1         |
| time/                   |           |
|    fps                  | 367       |
|    iterations           | 33        |
|    time_elapsed         | 184       |
|    total_timesteps      | 67584     |
| train/                  |           |
|    approx_kl            | 0.0577729 |
|    clip_fraction        | 0.275     |
|    clip_range           | 0.2       |
|    entropy_loss         | 0.132     |
|    explained_variance   | -110      |
|    learning_rate        | 0.0003    |
|    loss                 | 0.0127    |
|    n_updates            | 7180      |
|    policy_gradient_loss | 0.00834   |
|    std                  | 0.227     |
|    value_loss           | 0.0122    |
---------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.24      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.06        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 350         |
|    iterations           | 43          |
|    time_elapsed         | 251         |
|    total_timesteps      | 88064       |
| train/                  |             |
|    approx_kl            | 0.022892784 |
|    clip_fraction        | 0.213       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.124       |
|    explained_variance   | -197        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.029      |
|    n_updates            | 7280        |
|    policy_gradient_loss | 0.00781     |
|    std                  | 0.226       |
|    value_loss           | 0.0164      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

In [26]:
eval_episode_length=100000
training_episode_length=100000

In [39]:
env.seed(42)
torch.manual_seed(42)

In [40]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    results_overall[p]=[]
    results_polys[p]=[]
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-100000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.94 +/- 7.75
Focus Polytopes:
mean_reward:0.58 +/- 20.51
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.58 +/- 20.51
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.94 +/- 7.75
Focus Polytopes:
mean_reward:0.72 +/- 16.75
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.80 +/- 14.15
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.48 +/- 22.82
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:-0.29 +/- 35.93


In [28]:
results_overall = {0.:[(0.94,7.75)],0.05:[(0.97,5.48)],0.1:[(0.94,7.75)],0.5:[(0.98,4.48)],0.9:[(0.97,5.48)],1.0:[(0.97,5.48)]}
results_polys = {0.:[(0.58,20.51)],0.05:[(0.58,20.51)],0.1:[(0.72,16.75)],0.5:[(0.8,14.15)],0.9:[(0.48,22.82)],1.0:[(-0.29,35.93)]}

In [43]:
env.seed(43)
torch.manual_seed(43)

In [44]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-100000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-200000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.96     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 330      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.22        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 279         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.028366089 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.17        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 218         |
|    iterations           | 11          |
|    time_elapsed         | 103         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.016817104 |
|    clip_fraction        | 0.193       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0688     |
|    explained_variance   | 0.00316     |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0171     |
|    n_updates            | 7450        |
|    policy_gradient_loss | 0.00404     |
|    std                  | 0.254       |
|    value_loss           | 487         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 224         |
|    iterations           | 20          |
|    time_elapsed         | 182         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.019194994 |
|    clip_fraction        | 0.204       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.00611    |
|    explained_variance   | -115        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.000295    |
|    n_updates            | 7540        |
|    policy_gradient_loss | 0.00501     |
|    std                  | 0.241       |
|    value_loss           | 0.052       |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.13       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 218        |
|    iterations           | 29         |
|    time_elapsed         | 271        |
|    total_timesteps      | 59392      |
| train/                  |            |
|    approx_kl            | 0.02382049 |
|    clip_fraction        | 0.226      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0424     |
|    explained_variance   | -201       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00492    |
|    n_updates            | 7630       |
|    policy_gradient_loss | 0.012      |
|    std                  | 0.238      |
|    value_loss           | 0.0223     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.23       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 221        |
|    iterations           | 38         |
|    time_elapsed         | 351        |
|    total_timesteps      | 77824      |
| train/                  |            |
|    approx_kl            | 0.02809542 |
|    clip_fraction        | 0.225      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0572     |
|    explained_variance   | 0.00162    |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00751    |
|    n_updates            | 7720       |
|    policy_gradient_loss | 0.00332    |
|    std                  | 0.236      |
|    value_loss           | 920        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.12        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 388         |
|    iterations           | 6           |
|    time_elapsed         | 31          |
|    total_timesteps      | 12288       |
| train/                  |             |
|    approx_kl            | 0.028850378 |
|    clip_fraction        | 0.236       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.122       |
|    explained_variance   | -30.7       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0183      |
|    n_updates            | 7400        |
|    policy_gradient_loss | 0.0144      |
|    std                  | 0.228       |
|    value_loss           | 0.00505     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.27       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 343        |
|    iterations           | 15         |
|    time_elapsed         | 89         |
|    total_timesteps      | 30720      |
| train/                  |            |
|    approx_kl            | 0.02317613 |
|    clip_fraction        | 0.213      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0443     |
|    explained_variance   | -10.5      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00192    |
|    n_updates            | 7490       |
|    policy_gradient_loss | 0.00482    |
|    std                  | 0.236      |
|    value_loss           | 0.0044     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.18        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 341         |
|    iterations           | 24          |
|    time_elapsed         | 143         |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.027266366 |
|    clip_fraction        | 0.227       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0428      |
|    explained_variance   | -7.19       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0353      |
|    n_updates            | 7580        |
|    policy_gradient_loss | 0.00997     |
|    std                  | 0.237       |
|    value_loss           | 0.0058      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.21        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 355         |
|    iterations           | 33          |
|    time_elapsed         | 190         |
|    total_timesteps      | 67584       |
| train/                  |             |
|    approx_kl            | 0.042420946 |
|    clip_fraction        | 0.233       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0846      |
|    explained_variance   | -46.3       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0631      |
|    n_updates            | 7670        |
|    policy_gradient_loss | 0.00354     |
|    std                  | 0.232       |
|    value_loss           | 0.0775      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.14        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 363         |
|    iterations           | 42          |
|    time_elapsed         | 236         |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.019846363 |
|    clip_fraction        | 0.235       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0651      |
|    explained_variance   | -551        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00655    |
|    n_updates            | 7760        |
|    policy_gradient_loss | -0.000486   |
|    std                  | 0.234       |
|    value_loss           | 0.0273      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

In [27]:
env.seed(1997)
torch.manual_seed(1997)

In [29]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-200000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.88 +/- 10.96
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.68 +/- 17.90
Focus Polytopes:
mean_reward:0.78 +/- 14.85
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.87 +/- 11.41
Focus Polytopes:
mean_reward:0.78 +/- 14.85
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.96 +/- 6.33
Focus Polytopes:
mean_reward:0.81 +/- 13.80
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.41 +/- 24.31
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.95 +/- 7.08
Focus Polytopes:
mean_reward:0.66 +/- 18.45


In [30]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-200000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-300000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.36     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 331      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.09        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 285         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.029406453 |
|    clip_fraction        | 0.267       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.14        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 252         |
|    iterations           | 11          |
|    time_elapsed         | 89          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.021458711 |
|    clip_fraction        | 0.203       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0373     |
|    explained_variance   | -44.4       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0266      |
|    n_updates            | 7940        |
|    policy_gradient_loss | 0.00509     |
|    std                  | 0.249       |
|    value_loss           | 0.00513     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.14        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 251         |
|    iterations           | 20          |
|    time_elapsed         | 162         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.029649748 |
|    clip_fraction        | 0.244       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.124      |
|    explained_variance   | 0.000557    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00854     |
|    n_updates            | 8030        |
|    policy_gradient_loss | 0.00273     |
|    std                  | 0.258       |
|    value_loss           | 489         |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.03        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 251         |
|    iterations           | 29          |
|    time_elapsed         | 236         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.020380562 |
|    clip_fraction        | 0.233       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0651     |
|    explained_variance   | -39.5       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0166      |
|    n_updates            | 8120        |
|    policy_gradient_loss | 0.00691     |
|    std                  | 0.252       |
|    value_loss           | 0.00636     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.18        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 251         |
|    iterations           | 38          |
|    time_elapsed         | 310         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.023252608 |
|    clip_fraction        | 0.208       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0284     |
|    explained_variance   | -77.3       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.004       |
|    n_updates            | 8210        |
|    policy_gradient_loss | 0.00534     |
|    std                  | 0.245       |
|    value_loss           | 0.00678     |
-----------------------------------------
----------------------------------------
| rollout/                |        

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.34        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 382         |
|    iterations           | 14          |
|    time_elapsed         | 75          |
|    total_timesteps      | 28672       |
| train/                  |             |
|    approx_kl            | 0.018172607 |
|    clip_fraction        | 0.208       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0869     |
|    explained_variance   | -11.6       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0257      |
|    n_updates            | 7970        |
|    policy_gradient_loss | -0.00259    |
|    std                  | 0.252       |
|    value_loss           | 0.00551     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.51        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 377         |
|    iterations           | 23          |
|    time_elapsed         | 124         |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.048064336 |
|    clip_fraction        | 0.195       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0723     |
|    explained_variance   | -21.7       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00582     |
|    n_updates            | 8060        |
|    policy_gradient_loss | 0.00287     |
|    std                  | 0.25        |
|    value_loss           | 0.00304     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.21       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 376        |
|    iterations           | 32         |
|    time_elapsed         | 174        |
|    total_timesteps      | 65536      |
| train/                  |            |
|    approx_kl            | 0.01799675 |
|    clip_fraction        | 0.183      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0711    |
|    explained_variance   | -106       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00392    |
|    n_updates            | 8150       |
|    policy_gradient_loss | 0.00441    |
|    std                  | 0.253      |
|    value_loss           | 0.0159     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.12        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 375         |
|    iterations           | 41          |
|    time_elapsed         | 223         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.025798582 |
|    clip_fraction        | 0.205       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0706     |
|    explained_variance   | -190        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0383      |
|    n_updates            | 8240        |
|    policy_gradient_loss | 0.0119      |
|    std                  | 0.25        |
|    value_loss           | 0.0085      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

--- 270.33099722862244 seconds ---


In [31]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-300000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:


/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:0.94 +/- 7.75
Focus Polytopes:
mean_reward:0.90 +/- 10.01
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.68 +/- 17.90
Focus Polytopes:
mean_reward:-0.83 +/- 42.78
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.85 +/- 12.26
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.96 +/- 6.33
Focus Polytopes:
mean_reward:0.90 +/- 10.01
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.60 +/- 20.02
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.86 +/- 11.84


In [32]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-300000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-400000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.12     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 337      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.19       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 285        |
|    iterations           | 2          |
|    time_elapsed         | 14         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02039217 |
|    clip_fraction        | 0.21       |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.0368     |
| 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.15        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 11          |
|    time_elapsed         | 88          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.020480003 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.077       |
|    explained_variance   | -2.38e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00664     |
|    n_updates            | 8430        |
|    policy_gradient_loss | 0.00565     |
|    std                  | 0.234       |
|    value_loss           | 489         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.3         |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 252         |
|    iterations           | 20          |
|    time_elapsed         | 162         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.021611203 |
|    clip_fraction        | 0.212       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0502      |
|    explained_variance   | -73.9       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00696    |
|    n_updates            | 8520        |
|    policy_gradient_loss | 0.00511     |
|    std                  | 0.236       |
|    value_loss           | 0.00792     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.03        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 251         |
|    iterations           | 29          |
|    time_elapsed         | 236         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.017580792 |
|    clip_fraction        | 0.185       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0391      |
|    explained_variance   | -23         |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0277     |
|    n_updates            | 8610        |
|    policy_gradient_loss | 0.00516     |
|    std                  | 0.236       |
|    value_loss           | 0.00322     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.08        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 250         |
|    iterations           | 38          |
|    time_elapsed         | 310         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.023746502 |
|    clip_fraction        | 0.221       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.0888      |
|    explained_variance   | -136        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00279    |
|    n_updates            | 8700        |
|    policy_gradient_loss | 0.00202     |
|    std                  | 0.232       |
|    value_loss           | 0.0117      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.09        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 381         |
|    iterations           | 14          |
|    time_elapsed         | 75          |
|    total_timesteps      | 28672       |
| train/                  |             |
|    approx_kl            | 0.027835991 |
|    clip_fraction        | 0.211       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.084       |
|    explained_variance   | -0.000417   |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0211     |
|    n_updates            | 8460        |
|    policy_gradient_loss | 0.00605     |
|    std                  | 0.232       |
|    value_loss           | 489         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.12        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 377         |
|    iterations           | 23          |
|    time_elapsed         | 124         |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.043502554 |
|    clip_fraction        | 0.233       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.087       |
|    explained_variance   | -96.1       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0478      |
|    n_updates            | 8550        |
|    policy_gradient_loss | 0.00343     |
|    std                  | 0.231       |
|    value_loss           | 1.94        |
-----------------------------------------
---------------------------------------
| rollout/                |         

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.06        |
|    ep_rew_mean          | -9.01       |
| time/                   |             |
|    fps                  | 376         |
|    iterations           | 32          |
|    time_elapsed         | 174         |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.018780971 |
|    clip_fraction        | 0.201       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.113       |
|    explained_variance   | -714        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0869      |
|    n_updates            | 8640        |
|    policy_gradient_loss | 0.0064      |
|    std                  | 0.23        |
|    value_loss           | 0.0837      |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.27        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 376         |
|    iterations           | 41          |
|    time_elapsed         | 223         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.021087792 |
|    clip_fraction        | 0.241       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.175       |
|    explained_variance   | -17.5       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.334       |
|    n_updates            | 8730        |
|    policy_gradient_loss | 0.013       |
|    std                  | 0.221       |
|    value_loss           | 0.627       |
-----------------------------------------
-----------------------------------------
| rollout/                |       

--- 268.57478427886963 seconds ---


In [33]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-400000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.96 +/- 6.33
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.85 +/- 12.26
Focus Polytopes:
mean_reward:0.71 +/- 17.04
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:0.90 +/- 10.01
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.95 +/- 7.08
Focus Polytopes:
mean_reward:-0.39 +/- 37.29
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.43 +/- 23.89
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.96 +/- 6.33
Focus Polytopes:
mean_reward:0.55 +/- 21.23


In [34]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-400000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-500000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.33     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 342      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.87        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 293         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.041977957 |
|    clip_fraction        | 0.219       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.1         |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 256         |
|    iterations           | 11          |
|    time_elapsed         | 87          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.017999526 |
|    clip_fraction        | 0.211       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.14        |
|    explained_variance   | -125        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00859     |
|    n_updates            | 8920        |
|    policy_gradient_loss | 0.00759     |
|    std                  | 0.226       |
|    value_loss           | 0.00566     |
-----------------------------------------
---------------------------------------
| rollout/                |         

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2           |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 20          |
|    time_elapsed         | 160         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.027323823 |
|    clip_fraction        | 0.226       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.157       |
|    explained_variance   | -18.7       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0312      |
|    n_updates            | 9010        |
|    policy_gradient_loss | 0.00508     |
|    std                  | 0.224       |
|    value_loss           | 0.0033      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.07        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 254         |
|    iterations           | 29          |
|    time_elapsed         | 233         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.019866224 |
|    clip_fraction        | 0.252       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.218       |
|    explained_variance   | -1.9        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0343      |
|    n_updates            | 9100        |
|    policy_gradient_loss | 0.00783     |
|    std                  | 0.217       |
|    value_loss           | 0.00512     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.16       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 254        |
|    iterations           | 38         |
|    time_elapsed         | 306        |
|    total_timesteps      | 77824      |
| train/                  |            |
|    approx_kl            | 0.02664064 |
|    clip_fraction        | 0.248      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.265      |
|    explained_variance   | -1.65      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0248    |
|    n_updates            | 9190       |
|    policy_gradient_loss | 0.00536    |
|    std                  | 0.212      |
|    value_loss           | 0.0104     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.04       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 399        |
|    iterations           | 7          |
|    time_elapsed         | 35         |
|    total_timesteps      | 14336      |
| train/                  |            |
|    approx_kl            | 0.03935688 |
|    clip_fraction        | 0.208      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.285      |
|    explained_variance   | -13.4      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0107    |
|    n_updates            | 8880       |
|    policy_gradient_loss | 0.0111     |
|    std                  | 0.21       |
|    value_loss           | 0.00386    |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.08        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 388         |
|    iterations           | 16          |
|    time_elapsed         | 84          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.017030636 |
|    clip_fraction        | 0.243       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.3         |
|    explained_variance   | -151        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0153      |
|    n_updates            | 8970        |
|    policy_gradient_loss | 0.00459     |
|    std                  | 0.21        |
|    value_loss           | 0.0307      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.23        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 384         |
|    iterations           | 25          |
|    time_elapsed         | 133         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.015646815 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.273       |
|    explained_variance   | -11.6       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0286      |
|    n_updates            | 9060        |
|    policy_gradient_loss | 0.0141      |
|    std                  | 0.213       |
|    value_loss           | 0.00277     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.16        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 382         |
|    iterations           | 34          |
|    time_elapsed         | 181         |
|    total_timesteps      | 69632       |
| train/                  |             |
|    approx_kl            | 0.024628788 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.316       |
|    explained_variance   | -6.12       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0222     |
|    n_updates            | 9150        |
|    policy_gradient_loss | 0.0103      |
|    std                  | 0.21        |
|    value_loss           | 0.00582     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.23        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 381         |
|    iterations           | 43          |
|    time_elapsed         | 230         |
|    total_timesteps      | 88064       |
| train/                  |             |
|    approx_kl            | 0.022897048 |
|    clip_fraction        | 0.204       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.314       |
|    explained_variance   | -3.43       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.013      |
|    n_updates            | 9240        |
|    policy_gradient_loss | 0.00516     |
|    std                  | 0.209       |
|    value_loss           | 0.00304     |
-----------------------------------------
----------------------------------------
| rollout/                |        

In [35]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-500000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.92 +/- 8.95
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.81 +/- 13.80
Focus Polytopes:
mean_reward:0.58 +/- 20.51
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:0.96 +/- 6.33
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:0.92 +/- 8.95
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:0.59 +/- 20.26
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.67 +/- 18.18


In [36]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-500000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-600000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.19     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 318      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.92        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 280         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.034706473 |
|    clip_fraction        | 0.267       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.22        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 260         |
|    iterations           | 11          |
|    time_elapsed         | 86          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.018063655 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.219       |
|    explained_variance   | -7.88       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0334      |
|    n_updates            | 9410        |
|    policy_gradient_loss | 0.00571     |
|    std                  | 0.218       |
|    value_loss           | 0.00204     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.97        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 258         |
|    iterations           | 20          |
|    time_elapsed         | 158         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.032392494 |
|    clip_fraction        | 0.2         |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.171       |
|    explained_variance   | -54.9       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0108      |
|    n_updates            | 9500        |
|    policy_gradient_loss | 0.0014      |
|    std                  | 0.221       |
|    value_loss           | 0.0172      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.98        |
|    ep_rew_mean          | -9.01       |
| time/                   |             |
|    fps                  | 256         |
|    iterations           | 29          |
|    time_elapsed         | 231         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.019535381 |
|    clip_fraction        | 0.225       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.245       |
|    explained_variance   | 0.00174     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0753      |
|    n_updates            | 9590        |
|    policy_gradient_loss | 0.006       |
|    std                  | 0.213       |
|    value_loss           | 487         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.07        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 255         |
|    iterations           | 38          |
|    time_elapsed         | 304         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.019477373 |
|    clip_fraction        | 0.236       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.236       |
|    explained_variance   | -6.5        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0439     |
|    n_updates            | 9680        |
|    policy_gradient_loss | 0.00761     |
|    std                  | 0.216       |
|    value_loss           | 0.000208    |
-----------------------------------------
----------------------------------------
| rollout/                |        

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.24        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 403         |
|    iterations           | 8           |
|    time_elapsed         | 40          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.019131979 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.307       |
|    explained_variance   | -11.6       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0408     |
|    n_updates            | 9380        |
|    policy_gradient_loss | 0.00863     |
|    std                  | 0.211       |
|    value_loss           | 0.00396     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.46       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 391        |
|    iterations           | 17         |
|    time_elapsed         | 88         |
|    total_timesteps      | 34816      |
| train/                  |            |
|    approx_kl            | 0.03432279 |
|    clip_fraction        | 0.233      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.302      |
|    explained_variance   | -14.8      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00702    |
|    n_updates            | 9470       |
|    policy_gradient_loss | 0.0101     |
|    std                  | 0.212      |
|    value_loss           | 0.00911    |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.36        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 388         |
|    iterations           | 26          |
|    time_elapsed         | 136         |
|    total_timesteps      | 53248       |
| train/                  |             |
|    approx_kl            | 0.020345833 |
|    clip_fraction        | 0.18        |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.354       |
|    explained_variance   | -11.4       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0111      |
|    n_updates            | 9560        |
|    policy_gradient_loss | 0.00746     |
|    std                  | 0.204       |
|    value_loss           | 0.00388     |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.99        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 387         |
|    iterations           | 35          |
|    time_elapsed         | 185         |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.033545382 |
|    clip_fraction        | 0.27        |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.313       |
|    explained_variance   | -31.6       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00279     |
|    n_updates            | 9650        |
|    policy_gradient_loss | 0.013       |
|    std                  | 0.212       |
|    value_loss           | 0.0054      |
-----------------------------------------
---------------------------------------
| rollout/                |         

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.21       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 387        |
|    iterations           | 44         |
|    time_elapsed         | 232        |
|    total_timesteps      | 90112      |
| train/                  |            |
|    approx_kl            | 0.05752319 |
|    clip_fraction        | 0.207      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.388      |
|    explained_variance   | -13.9      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0148     |
|    n_updates            | 9740       |
|    policy_gradient_loss | 0.0118     |
|    std                  | 0.201      |
|    value_loss           | 0.00272    |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

In [37]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-600000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.04 +/- 31.00
Focus Polytopes:
mean_reward:0.70 +/- 17.34
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.90 +/- 10.01
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.95 +/- 7.08
Focus Polytopes:
mean_reward:0.71 +/- 17.04
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:0.82 +/- 13.43
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.90 +/- 10.01
Focus Polytopes:
mean_reward:0.47 +/- 23.04


In [38]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-600000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-700000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.83     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 337      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 290         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.023266051 |
|    clip_fraction        | 0.182       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.92        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 258         |
|    iterations           | 11          |
|    time_elapsed         | 86          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.024024628 |
|    clip_fraction        | 0.188       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.249       |
|    explained_variance   | -12         |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0381      |
|    n_updates            | 9900        |
|    policy_gradient_loss | 0.0151      |
|    std                  | 0.213       |
|    value_loss           | 0.00275     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.93       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 256        |
|    iterations           | 20         |
|    time_elapsed         | 159        |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.04065465 |
|    clip_fraction        | 0.24       |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.204      |
|    explained_variance   | -0.827     |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0596     |
|    n_updates            | 9990       |
|    policy_gradient_loss | 0.00734    |
|    std                  | 0.218      |
|    value_loss           | 0.00182    |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.8         |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 252         |
|    iterations           | 29          |
|    time_elapsed         | 235         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.021637075 |
|    clip_fraction        | 0.203       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.26        |
|    explained_variance   | -0.498      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0418      |
|    n_updates            | 10080       |
|    policy_gradient_loss | 0.00949     |
|    std                  | 0.213       |
|    value_loss           | 0.002       |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.21       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 253        |
|    iterations           | 38         |
|    time_elapsed         | 307        |
|    total_timesteps      | 77824      |
| train/                  |            |
|    approx_kl            | 0.01387507 |
|    clip_fraction        | 0.172      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.307      |
|    explained_variance   | -0.000104  |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0541     |
|    n_updates            | 10170      |
|    policy_gradient_loss | 0.00439    |
|    std                  | 0.209      |
|    value_loss           | 488        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.28        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 397         |
|    iterations           | 14          |
|    time_elapsed         | 72          |
|    total_timesteps      | 28672       |
| train/                  |             |
|    approx_kl            | 0.032900944 |
|    clip_fraction        | 0.258       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.357       |
|    explained_variance   | -18         |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0284     |
|    n_updates            | 9930        |
|    policy_gradient_loss | 0.0046      |
|    std                  | 0.205       |
|    value_loss           | 0.00596     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.4         |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 392         |
|    iterations           | 23          |
|    time_elapsed         | 120         |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.019211885 |
|    clip_fraction        | 0.228       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.367       |
|    explained_variance   | -33.8       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00436     |
|    n_updates            | 10020       |
|    policy_gradient_loss | 0.00264     |
|    std                  | 0.206       |
|    value_loss           | 0.00467     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 389         |
|    iterations           | 32          |
|    time_elapsed         | 168         |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.015592384 |
|    clip_fraction        | 0.243       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.386       |
|    explained_variance   | -55.8       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0153     |
|    n_updates            | 10110       |
|    policy_gradient_loss | 0.0163      |
|    std                  | 0.205       |
|    value_loss           | 0.00527     |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.13        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 387         |
|    iterations           | 41          |
|    time_elapsed         | 216         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.028692843 |
|    clip_fraction        | 0.179       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.416       |
|    explained_variance   | 3.59e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.455       |
|    n_updates            | 10200       |
|    policy_gradient_loss | 0.00544     |
|    std                  | 0.198       |
|    value_loss           | 2.17e+03    |
-----------------------------------------
----------------------------------------
| rollout/                |        

--- 268.07624340057373 seconds ---


In [39]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-700000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.98 +/- 4.48
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.23 +/- 27.77
Focus Polytopes:
mean_reward:0.92 +/- 8.95
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.96 +/- 6.33
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.98 +/- 4.48
Focus Polytopes:
mean_reward:0.91 +/- 9.50
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.80 +/- 14.15
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.95 +/- 7.08
Focus Polytopes:
mean_reward:0.67 +/- 18.18


In [47]:
results_overall

{0.0: [(0.94, 7.75),
  (0.97998, 4.476563324649836),
  (0.93994, 7.7534800442382),
  (0.97998, 4.476563324649836),
  (0.91992, 8.952858046099026),
  (0.98999, 3.16542411058929),
  (0.98999, 3.1654241105892895)],
 0.05: [(0.97, 5.48),
  (0.67968, 17.903567105401088),
  (0.67968, 17.903567105401088),
  (0.84985, 12.258776650934628),
  (0.80981, 13.796521944457595),
  (0.03904, 30.999959933496687),
  (0.22923, 27.765926665737272)],
 0.1: [(0.94, 7.75),
  (0.86987, 11.412414125990173),
  (0.97998, 4.476563324649836),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.97998, 4.476563324649836),
  (0.98999, 3.1654241105892895)],
 0.5: [(0.98, 4.48),
  (0.95996, 6.330753256793382),
  (0.95996, 6.330753256793382),
  (0.94995, 7.077961923993376),
  (1.0, 0.0),
  (0.94995, 7.077961923993375),
  (0.97998, 4.476563324649836)],
 0.9: [(0.97, 5.48),
  (0.96997, 5.4826205594678905),
  (0.98999, 3.1654241105892904),
  (0.98999, 3.1654241105892895),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.98999, 3.1654241105892895)],
 1.0: [(0

In [48]:
results_polys

{0.0: [(0.58, 20.51),
  (0.87988, 10.964747657178437),
  (0.8999, 10.009499487486877),
  (0.95996, 6.330753256793382),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.97998, 4.476563324649836)],
 0.05: [(0.58, 20.51),
  (0.77978, 14.845596086099071),
  (-0.83183, 42.78207836058342),
  (0.70971, 17.04394384278181),
  (0.57958, 20.510086957972653),
  (0.6997, 17.335227714397067),
  (0.91992, 8.952858046099026)],
 0.1: [(0.72, 16.75),
  (0.77978, 14.845596086099071),
  (0.84985, 12.258776650934628),
  (0.8999, 10.009499487486877),
  (0.95996, 6.330753256793382),
  (0.8999, 10.009499487486876),
  (0.95996, 6.330753256793382)],
 0.5: [(0.8, 14.15),
  (0.80981, 13.796521944457595),
  (0.8999, 10.009499487486876),
  (-0.39139, 37.294040058270696),
  (0.91992, 8.952858046099026),
  (0.70971, 17.043943842781815),
  (0.90991, 9.495892469478584)],
 0.9: [(0.48, 22.82),
  (0.40941, 24.307031769673152),
  (0.5996, 20.0159955995199),
  (0.42943, 23.89173538852086),
  (0.58959, 20.264549677500856),
  (0.81982, 13.4

In [11]:
results_overall={0.0: [(0.94, 7.75),
  (0.97998, 4.476563324649836),
  (0.93994, 7.7534800442382),
  (0.97998, 4.476563324649836),
  (0.91992, 8.952858046099026),
  (0.98999, 3.16542411058929),
  (0.98999, 3.1654241105892895)],
 0.05: [(0.97, 5.48),
  (0.67968, 17.903567105401088),
  (0.67968, 17.903567105401088),
  (0.84985, 12.258776650934628),
  (0.80981, 13.796521944457595),
  (0.03904, 30.999959933496687),
  (0.22923, 27.765926665737272)],
 0.1: [(0.94, 7.75),
  (0.86987, 11.412414125990173),
  (0.97998, 4.476563324649836),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.97998, 4.476563324649836),
  (0.98999, 3.1654241105892895)],
 0.5: [(0.98, 4.48),
  (0.95996, 6.330753256793382),
  (0.95996, 6.330753256793382),
  (0.94995, 7.077961923993376),
  (1.0, 0.0),
  (0.94995, 7.077961923993375),
  (0.97998, 4.476563324649836)],
 0.9: [(0.97, 5.48),
  (0.96997, 5.4826205594678905),
  (0.98999, 3.1654241105892904),
  (0.98999, 3.1654241105892895),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.98999, 3.1654241105892895)],
 1.0: [(0.97, 5.48),
  (0.94995, 7.077961923993375),
  (0.96997, 5.4826205594678905),
  (0.95996, 6.330753256793382),
  (0.96997, 5.4826205594678905),
  (0.8999, 10.009499487486877),
  (0.94995, 7.077961923993376)]}
results_polys={0.0: [(0.58, 20.51),
  (0.87988, 10.964747657178437),
  (0.8999, 10.009499487486877),
  (0.95996, 6.330753256793382),
  (1.0, 0.0),
  (1.0, 0.0),
  (0.97998, 4.476563324649836)],
 0.05: [(0.58, 20.51),
  (0.77978, 14.845596086099071),
  (-0.83183, 42.78207836058342),
  (0.70971, 17.04394384278181),
  (0.57958, 20.510086957972653),
  (0.6997, 17.335227714397067),
  (0.91992, 8.952858046099026)],
 0.1: [(0.72, 16.75),
  (0.77978, 14.845596086099071),
  (0.84985, 12.258776650934628),
  (0.8999, 10.009499487486877),
  (0.95996, 6.330753256793382),
  (0.8999, 10.009499487486876),
  (0.95996, 6.330753256793382)],
 0.5: [(0.8, 14.15),
  (0.80981, 13.796521944457595),
  (0.8999, 10.009499487486876),
  (-0.39139, 37.294040058270696),
  (0.91992, 8.952858046099026),
  (0.70971, 17.043943842781815),
  (0.90991, 9.495892469478584)],
 0.9: [(0.48, 22.82),
  (0.40941, 24.307031769673152),
  (0.5996, 20.0159955995199),
  (0.42943, 23.89173538852086),
  (0.58959, 20.264549677500856),
  (0.81982, 13.428615534283496),
  (0.7998, 14.154862060790277)],
 1.0: [(-0.29, 35.93),
  (0.65966, 18.45438995698314),
  (0.85986, 11.843162617324818),
  (0.54955, 21.2296383576711),
  (0.66967, 18.18106740791365),
  (0.46947, 23.03864292702806),
  (0.66967, 18.18106740791365)]}

In [12]:
eval_episode_length=100000
training_episode_length=100000
env.seed(2022)
torch.manual_seed(2022)

In [13]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-700000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-800000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.81     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 289      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.98        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 253         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.031546764 |
|    clip_fraction        | 0.214       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.99        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 251         |
|    iterations           | 11          |
|    time_elapsed         | 89          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.018945675 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.227       |
|    explained_variance   | -9.31       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00678     |
|    n_updates            | 10390       |
|    policy_gradient_loss | 0.00213     |
|    std                  | 0.217       |
|    value_loss           | 0.0016      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.91        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 256         |
|    iterations           | 20          |
|    time_elapsed         | 159         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.018699173 |
|    clip_fraction        | 0.167       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.23        |
|    explained_variance   | -4.32       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.000358    |
|    n_updates            | 10480       |
|    policy_gradient_loss | 0.00353     |
|    std                  | 0.216       |
|    value_loss           | 0.00258     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.04        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 254         |
|    iterations           | 29          |
|    time_elapsed         | 233         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.018536408 |
|    clip_fraction        | 0.214       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.194       |
|    explained_variance   | 0.00446     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0332      |
|    n_updates            | 10570       |
|    policy_gradient_loss | -0.00264    |
|    std                  | 0.22        |
|    value_loss           | 487         |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.79        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 245         |
|    iterations           | 38          |
|    time_elapsed         | 316         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.017131012 |
|    clip_fraction        | 0.196       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.218       |
|    explained_variance   | -90.5       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0114      |
|    n_updates            | 10660       |
|    policy_gradient_loss | 0.0037      |
|    std                  | 0.218       |
|    value_loss           | 0.00561     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.08        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 368         |
|    iterations           | 7           |
|    time_elapsed         | 38          |
|    total_timesteps      | 14336       |
| train/                  |             |
|    approx_kl            | 0.047608532 |
|    clip_fraction        | 0.237       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.598       |
|    explained_variance   | -14.6       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00585    |
|    n_updates            | 10350       |
|    policy_gradient_loss | 0.000434    |
|    std                  | 0.179       |
|    value_loss           | 0.00601     |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.16        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 317         |
|    iterations           | 16          |
|    time_elapsed         | 103         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.031840414 |
|    clip_fraction        | 0.245       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.538       |
|    explained_variance   | -139        |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0191     |
|    n_updates            | 10440       |
|    policy_gradient_loss | 0.0145      |
|    std                  | 0.187       |
|    value_loss           | 0.00301     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.96        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 314         |
|    iterations           | 25          |
|    time_elapsed         | 162         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.030737244 |
|    clip_fraction        | 0.192       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.434       |
|    explained_variance   | -3.96       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.000563    |
|    n_updates            | 10530       |
|    policy_gradient_loss | 0.00569     |
|    std                  | 0.195       |
|    value_loss           | 0.00275     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.21       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 325        |
|    iterations           | 34         |
|    time_elapsed         | 213        |
|    total_timesteps      | 69632      |
| train/                  |            |
|    approx_kl            | 0.01729393 |
|    clip_fraction        | 0.212      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.465      |
|    explained_variance   | -29.8      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0433    |
|    n_updates            | 10620      |
|    policy_gradient_loss | 0.00615    |
|    std                  | 0.193      |
|    value_loss           | 0.00756    |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.21        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 333         |
|    iterations           | 43          |
|    time_elapsed         | 263         |
|    total_timesteps      | 88064       |
| train/                  |             |
|    approx_kl            | 0.024311077 |
|    clip_fraction        | 0.239       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.45        |
|    explained_variance   | -24.4       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00465     |
|    n_updates            | 10710       |
|    policy_gradient_loss | 0.0014      |
|    std                  | 0.196       |
|    value_loss           | 0.0101      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

In [14]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-800000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:


/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.99 +/- 3.17
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.35 +/- 25.51
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.95 +/- 7.08
Focus Polytopes:
mean_reward:0.98 +/- 4.48
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.99 +/- 3.17
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.96 +/- 6.33
Focus Polytopes:
mean_reward:0.77 +/- 15.18
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.86 +/- 11.84
Focus Polytopes:
mean_reward:0.34 +/- 25.71


In [15]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-800000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-900000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2.03     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 190      |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.87        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 175         |
|    iterations           | 2           |
|    time_elapsed         | 23          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.031140193 |
|    clip_fraction        | 0.19        |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.02        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 207         |
|    iterations           | 11          |
|    time_elapsed         | 108         |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.013727346 |
|    clip_fraction        | 0.193       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.205       |
|    explained_variance   | -3.23       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0241      |
|    n_updates            | 10880       |
|    policy_gradient_loss | 0.00161     |
|    std                  | 0.218       |
|    value_loss           | 0.00295     |
-----------------------------------------
---------------------------------------
| rollout/                |         

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.11        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 215         |
|    iterations           | 20          |
|    time_elapsed         | 190         |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.015694097 |
|    clip_fraction        | 0.206       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.279       |
|    explained_variance   | -47.4       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0156      |
|    n_updates            | 10970       |
|    policy_gradient_loss | 0.00861     |
|    std                  | 0.211       |
|    value_loss           | 0.000751    |
-----------------------------------------
----------------------------------------
| rollout/                |        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.12        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 219         |
|    iterations           | 29          |
|    time_elapsed         | 270         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.029419012 |
|    clip_fraction        | 0.208       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.381       |
|    explained_variance   | -26.1       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0254     |
|    n_updates            | 11060       |
|    policy_gradient_loss | 0.00799     |
|    std                  | 0.199       |
|    value_loss           | 0.00103     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.03        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 217         |
|    iterations           | 38          |
|    time_elapsed         | 357         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.017380767 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.278       |
|    explained_variance   | -12.5       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0123     |
|    n_updates            | 11150       |
|    policy_gradient_loss | 0.00655     |
|    std                  | 0.21        |
|    value_loss           | 0.00196     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.17        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 413         |
|    iterations           | 7           |
|    time_elapsed         | 34          |
|    total_timesteps      | 14336       |
| train/                  |             |
|    approx_kl            | 0.020117857 |
|    clip_fraction        | 0.191       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.474       |
|    explained_variance   | -37.7       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00634     |
|    n_updates            | 10840       |
|    policy_gradient_loss | 0.0013      |
|    std                  | 0.193       |
|    value_loss           | 0.0128      |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.07       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 403        |
|    iterations           | 16         |
|    time_elapsed         | 81         |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.04240513 |
|    clip_fraction        | 0.244      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.522      |
|    explained_variance   | -108       |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0154     |
|    n_updates            | 10930      |
|    policy_gradient_loss | 0.0188     |
|    std                  | 0.187      |
|    value_loss           | 0.0051     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.15        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 400         |
|    iterations           | 25          |
|    time_elapsed         | 127         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.029164337 |
|    clip_fraction        | 0.259       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.562       |
|    explained_variance   | -116        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00276     |
|    n_updates            | 11020       |
|    policy_gradient_loss | 0.00566     |
|    std                  | 0.185       |
|    value_loss           | 0.017       |
-----------------------------------------
----------------------------------------
| rollout/                |        

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.15       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 397        |
|    iterations           | 35         |
|    time_elapsed         | 180        |
|    total_timesteps      | 71680      |
| train/                  |            |
|    approx_kl            | 0.03285235 |
|    clip_fraction        | 0.203      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.57       |
|    explained_variance   | -30.3      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0258    |
|    n_updates            | 11120      |
|    policy_gradient_loss | 0.00608    |
|    std                  | 0.183      |
|    value_loss           | 0.00469    |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.18       |
|    ep_rew_mean          | -9.01      |
| time/                   |            |
|    fps                  | 397        |
|    iterations           | 44         |
|    time_elapsed         | 226        |
|    total_timesteps      | 90112      |
| train/                  |            |
|    approx_kl            | 0.03711408 |
|    clip_fraction        | 0.225      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.622      |
|    explained_variance   | -13.3      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.00465   |
|    n_updates            | 11210      |
|    policy_gradient_loss | 0.0127     |
|    std                  | 0.178      |
|    value_loss           | 0.00404    |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

In [16]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-900000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.98 +/- 4.48
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.47 +/- 23.04
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.85 +/- 12.26
Focus Polytopes:
mean_reward:0.99 +/- 3.17
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.98 +/- 4.48
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.99 +/- 3.17
Focus Polytopes:
mean_reward:0.86 +/- 11.84
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.90 +/- 10.01
Focus Polytopes:
mean_reward:0.98 +/- 4.48


In [17]:
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-900000-"+str(p))
    model.set_env(env)
    
    print("p=",p)

    env.init_polytopes(p,retrain_polytopes)
    start_time = time.time()
    model=model.learn(total_timesteps=training_episode_length)
    print("--- %s seconds ---" % (time.time() - start_time))

    model.save("model_backup/zeppelin-avoidance-windsystem-small2-1400000-1000000-"+str(p))

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.81     |
|    ep_rew_mean     | 1        |
| time/              |          |
|    fps             | 304      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.01        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 269         |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016252391 |
|    clip_fraction        | 0.21        |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.01        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 245         |
|    iterations           | 11          |
|    time_elapsed         | 91          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.019787671 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.212       |
|    explained_variance   | -0.737      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00761     |
|    n_updates            | 11370       |
|    policy_gradient_loss | 0.00673     |
|    std                  | 0.217       |
|    value_loss           | 0.00123     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.85       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 242        |
|    iterations           | 20         |
|    time_elapsed         | 169        |
|    total_timesteps      | 40960      |
| train/                  |            |
|    approx_kl            | 0.02297139 |
|    clip_fraction        | 0.182      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.275      |
|    explained_variance   | -1.88      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.00694    |
|    n_updates            | 11460      |
|    policy_gradient_loss | 0.00553    |
|    std                  | 0.211      |
|    value_loss           | 0.0032     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.99        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 237         |
|    iterations           | 29          |
|    time_elapsed         | 250         |
|    total_timesteps      | 59392       |
| train/                  |             |
|    approx_kl            | 0.036941633 |
|    clip_fraction        | 0.238       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.261       |
|    explained_variance   | -0.758      |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0223     |
|    n_updates            | 11550       |
|    policy_gradient_loss | 0.00936     |
|    std                  | 0.212       |
|    value_loss           | 0.002       |
-----------------------------------------
---------------------------------------
| rollout/                |         

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.15        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 236         |
|    iterations           | 38          |
|    time_elapsed         | 329         |
|    total_timesteps      | 77824       |
| train/                  |             |
|    approx_kl            | 0.020359876 |
|    clip_fraction        | 0.199       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.245       |
|    explained_variance   | -8.57       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.000654   |
|    n_updates            | 11640       |
|    policy_gradient_loss | 0.00311     |
|    std                  | 0.214       |
|    value_loss           | 0.00366     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.51       |
|    ep_rew_mean          | 1          |
| time/                   |            |
|    fps                  | 334        |
|    iterations           | 7          |
|    time_elapsed         | 42         |
|    total_timesteps      | 14336      |
| train/                  |            |
|    approx_kl            | 0.03737913 |
|    clip_fraction        | 0.278      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.74       |
|    explained_variance   | -193       |
|    learning_rate        | 0.0003     |
|    loss                 | -0.012     |
|    n_updates            | 11330      |
|    policy_gradient_loss | 0.00793    |
|    std                  | 0.168      |
|    value_loss           | 0.0158     |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_me

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.14        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 314         |
|    iterations           | 16          |
|    time_elapsed         | 104         |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.021696616 |
|    clip_fraction        | 0.189       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.711       |
|    explained_variance   | -9.31       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.015      |
|    n_updates            | 11420       |
|    policy_gradient_loss | 0.00224     |
|    std                  | 0.17        |
|    value_loss           | 0.00598     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.07        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 307         |
|    iterations           | 25          |
|    time_elapsed         | 166         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.028175518 |
|    clip_fraction        | 0.209       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.704       |
|    explained_variance   | -36.3       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0493      |
|    n_updates            | 11510       |
|    policy_gradient_loss | 0.00362     |
|    std                  | 0.171       |
|    value_loss           | 0.00961     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.34        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 308         |
|    iterations           | 34          |
|    time_elapsed         | 225         |
|    total_timesteps      | 69632       |
| train/                  |             |
|    approx_kl            | 0.036582828 |
|    clip_fraction        | 0.231       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.645       |
|    explained_variance   | -50         |
|    learning_rate        | 0.0003      |
|    loss                 | 0.344       |
|    n_updates            | 11600       |
|    policy_gradient_loss | 0.0139      |
|    std                  | 0.175       |
|    value_loss           | 0.00714     |
-----------------------------------------
-----------------------------------------
| rollout/                |       

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.15        |
|    ep_rew_mean          | 1           |
| time/                   |             |
|    fps                  | 294         |
|    iterations           | 43          |
|    time_elapsed         | 298         |
|    total_timesteps      | 88064       |
| train/                  |             |
|    approx_kl            | 0.024219107 |
|    clip_fraction        | 0.259       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.692       |
|    explained_variance   | -69.4       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0211     |
|    n_updates            | 11690       |
|    policy_gradient_loss | 0.00345     |
|    std                  | 0.171       |
|    value_loss           | 0.0157      |
-----------------------------------------
---------------------------------------
| rollout/                |         

In [18]:
# Performance of models on focus polytopes only?
for p in [0.0, 0.05, 0.1, 0.5, 0.9, 1.0]:
    model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-1000000-"+str(p))
    model.set_env(env)
    print("p=",p)
    
    print("Overall:")
    env.init_polytopes(1.0,[])
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_overall[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    print("Focus Polytopes:")
    env.init_polytopes(0.0,retrain_polytopes)
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
    results_polys[p].append((mean_reward, std_reward))
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.0
Overall:
mean_reward:0.96 +/- 6.33
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.05
Overall:
mean_reward:0.85 +/- 12.26
Focus Polytopes:
mean_reward:0.99 +/- 3.17
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.1
Overall:
mean_reward:0.51 +/- 22.15
Focus Polytopes:
mean_reward:0.99 +/- 3.17
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.5
Overall:
mean_reward:1.00 +/- 0.00
Focus Polytopes:
mean_reward:1.00 +/- 0.00
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 0.9
Overall:
mean_reward:0.97 +/- 5.48
Focus Polytopes:
mean_reward:0.76 +/- 15.51
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
p= 1.0
Overall:
mean_reward:0.92 +/- 8.95
Focus Polytopes:
mean_reward:0.83 +/- 13.05


## After retraining

In [9]:
eval_episode_length=100000
training_episode_length=100000

In [10]:
env.seed(42)
torch.manual_seed(42)

In [11]:
retrain_polytopes = None
with open("zeppelin-1400000-retrain-1000000-0.5-polytopes.pickle","rb") as f:
    retrain_polytopes = pickle.load(f)

In [12]:
# Total volume of polytopes?
# Share of instances usually (i.e. without "focus polytopes")

# Volume of state space:
N = 100000
n=4
l_b = np.array([env.unwrapped.MIN_X,env.unwrapped.MIN_Y,env.unwrapped.MIN_C,env.unwrapped.MAX_VELOCITY-env.unwrapped.MAX_TURBULENCE+0.1])
u_b = np.array([env.unwrapped.MAX_X,env.unwrapped.MAX_Y,env.unwrapped.MAX_C,env.unwrapped.MAX_WIND_SPEED])
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if not env.unwrapped.is_crash(x) and not env.unwrapped.reached_goal(x) and env.unwrapped.is_in_bounds(x):
        s+=1
print("s: ",s)
total_vol = np.prod(u_b - l_b) * (s / N)
print("total volume: ", total_vol)

poly_region = pc.Region(retrain_polytopes)
l_b, u_b = poly_region.bounding_box
l_b = l_b.flatten()
u_b = u_b.flatten()
print(l_b,",",u_b)
xs = env.unwrapped.np_random.uniform(low=l_b,high=u_b,size=(N,n))
s = 0
for x in xs:
    if x in poly_region:
        if not env.unwrapped.is_crash(x) and not env.unwrapped.reached_goal(x) and env.unwrapped.is_in_bounds(x):
                s+=1
poly_vol = np.prod(u_b - l_b) * (s / N)
print("poly volume: ", poly_vol)

# We only have an upper bound for the share, since polytopes may be partially outside the state space of interest
poly_share = poly_vol/total_vol
print("max share: ", poly_share)

s:  4512
total volume:  50332262.4
[-146.37511  -97.5       10.         5.     ] , [400. 400.  80.  30.]
poly volume:  35609990.77787027
max share:  0.7074983138026052


In [ ]:
model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-1000000-0.5")
model.set_env(env)

env.init_polytopes(0.0,retrain_polytopes)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=eval_episode_length)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


In [19]:
model = PPO.load("model_backup/zeppelin-avoidance-windsystem-small2-1400000-1000000-0.5")
model.set_env(env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [21]:
env.unwrapped.INCLUDE_UNWINNABLE = False
env.init_polytopes(0.0,retrain_polytopes)
#orig_observation = env.reset()
for i_episode in range(100):
    observation = env.reset()
    #env.unwrapped.state=[4.545,13,10,5.5]
    #env.unwrapped.state = [167.10239, -10.34823,  79.3649,    5.99876]
    #env.unwrapped.state = orig_observation
    observation = env.unwrapped.state
    total_reward=0
    for t in range(1000):
        print(observation)
        action, _states = model.predict(observation,deterministic=True)
        print([action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)])
        #print[[action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)]]
        #x1_norm = observation[0]/np.sqrt(observation[0]**2+observation[1]**2)
        #x2_norm = observation[1]/np.sqrt(observation[0]**2+observation[1]**2)
        #f = 1.0 if (x2_norm) < 0 else -1.0
        #action = [f, f*(0-x1_norm)]
        observation, reward, done, info = env.step(action)
        total_reward=0.99*total_reward+reward
        env.render()
        time.sleep(0.2)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Reward: ", total_reward)
            break
env.close()

[ 69.90218 -61.54047  49.81218  10.69485]
[1.0, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[-91.02743  -1.26261  64.68677  28.64434]
[-1.0, -0.0]
[-100.78373  -14.21089   64.68677   28.64434]
[-1.0, -0.0]
Episode finished after 2 timesteps
Reward:  1.0
[ 22.9063  -35.31046  35.30115  10.49014]
[0.2238152, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[-66.35822 -41.74392  32.21763  27.21313]
[-1.0, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[ 45.00846 -68.21664  58.33325  25.08126]
[0.39631796, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[ 32.37697 -47.3785   42.63395  23.36719]
[0.09334865, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[-102.74506    7.67821   70.81089   21.19506]
[-1.0, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[-118.38648  -39.77077   46.2788     7.19695]
[-1.0, -0.0]
[-132.37303  -41.12658   46.2788     7.19695]
[-1.0, -0.0]
[-140.38629  -47.81653   46.2788     7.19695]
[-1.0, -0.0]
Episode finished after 

[-84.54839  -5.40875  57.36565  18.2947 ]
[-1.0, -0.0]
[-93.98117  -8.25805  57.36565  18.2947 ]
[-1.0, -0.0]
Episode finished after 3 timesteps
Reward:  1.0
[-91.50156  26.08675  63.25625  13.45368]
[-1.0, -0.0]
[-97.57281  24.3559   63.25625  13.45368]
[-1.0, -0.0]
[-105.766     15.34429   63.25625   13.45368]
[-1.0, -0.0]
Episode finished after 3 timesteps
Reward:  1.0
[-67.36741 -49.71029  45.93739  28.27703]
[-0.93687975, -0.349651721603592]
[-75.30053 -60.53092  45.93739  28.27703]
[-0.9667293, -0.255801666817875]
Episode finished after 2 timesteps
Reward:  1.0
[-146.90766   -9.47057   79.9122     7.58837]
[-1.0, -0.0]
[-153.19368  -13.20897   79.9122     7.58837]
[-1.0, -0.0]
[-162.94532  -16.22664   79.9122     7.58837]
[-1.0, -0.0]
[-173.05322  -19.24465   79.9122     7.58837]
[-1.0, -0.0]
Episode finished after 4 timesteps
Reward:  1.0
[ 72.65759 -65.63762  51.60282  10.58289]
[1.0, -0.0]
Episode finished after 1 timesteps
Reward:  1.0
[-146.7409   -12.19247   79.28306    8.5

In [ ]:
env.unwrapped.INCLUDE_UNWINNABLE = False
env.init_polytopes(1.0,[])
#orig_observation = env.reset()
for i_episode in range(10):
    observation = env.reset()
    #env.unwrapped.state=[4.545,13,10,5.5]
    #env.unwrapped.state = orig_observation
    observation = env.unwrapped.state
    total_reward=0
    for t in range(1000):
        print(observation)
        action, _states = model.predict(observation,deterministic=True)
        print([action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)])
        #print[[action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)]]
        #x1_norm = observation[0]/np.sqrt(observation[0]**2+observation[1]**2)
        #x2_norm = observation[1]/np.sqrt(observation[0]**2+observation[1]**2)
        #f = 1.0 if (x2_norm) < 0 else -1.0
        #action = [f, f*(0-x1_norm)]
        observation, reward, done, info = env.step(action)
        total_reward=0.99*total_reward+reward
        env.render()
        time.sleep(0.5)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Reward: ", total_reward)
            time.sleep(5.0)
            break
env.close()

In [ ]:
env.unwrapped.INCLUDE_UNWINNABLE = False
env.init_polytopes(0.0,retrain_polytopes_certain)
for i_episode in range(100000):
    observation = env.reset()
    observation = env.unwrapped.state
    total_reward=0
    for t in range(1000):
        print(observation)
        action, _states = model.predict(observation,deterministic=True)
        print([action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)])
        observation, reward, done, info = env.step(action)
        assert reward >= 0, f"reward: {reward}"
        total_reward=0.99*total_reward+reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Reward: ", total_reward)
            break
env.close()

In [ ]:
env.unwrapped.INCLUDE_UNWINNABLE = False
bounds = pc.box2poly([[-100.,100.],[-100.,400.],[49.9,50.1],[24.9,25.1]])
env.init_polytopes(0.0,[bounds])
for i_episode in range(100000):
    observation = env.reset()
    observation = env.unwrapped.state
    total_reward=0
    for t in range(1000):
        print(observation)
        action, _states = model.predict(observation,deterministic=True)
        print([action[1]*action[0],action[1]*np.sqrt(1-action[0]**2)])
        observation, reward, done, info = env.step(action)
        assert reward >= 0, f"reward: {reward}"
        total_reward=0.99*total_reward+reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Reward: ", total_reward)
            break
env.close()

In [ ]:
x = [  3.55826, 253.8157,   24.94608,  50.0919 ]
print("certain")
for p in retrain_polytopes_certain:
    if x in p:
        print("found p")
print("uncertain")
for p in retrain_polytopes_uncertain:
    if x in p:
        print("found p")